In [33]:
import pandas as pd
import numpy as np

In [34]:
machines = pd.read_excel("data.xlsx", sheet_name="Máy")
machines.drop(columns=machines.filter(regex="STT").columns, inplace=True)
machines.rename(columns={"KÝ HIỆU": "id", "TÊN MÁY": "name", "TỐC ĐỘ\n(m/p)": "speed", "THỜI GIAN SET UP (p)": "set_up_time"}, inplace=True)
machines

,id,name,speed,set_up_time
0,R/S,Flexirapid 600,20,20
1,M/S,Twin 800,10,20
2,R/O,Corino,6,15
3,W,Aqua,20,15
4,ST,Brueckner,42,15
5,SI,Osthoff,60,10
6,CM,NaN,20,1
7,D,Thermoduplex,18,15
8,KD,Corino,25,20
9,P,Formula 1,17,20


In [35]:
data = pd.read_excel("data.xlsx")
sched_date = data["NGÀY ĐIỀU ĐỘ"][0]
data["QUY TRÌNH GIA CÔNG"] = [
    [machines[machines['id'] == step].index[0] for step in seq if not pd.isna(step)]
    for seq in data.filter(regex="^(QUY TRÌNH GIA CÔNG|Unnamed.*)").values.tolist()
]
data["deadline"] = (
    data["NGÀY TỚI HẠN"]
    .apply(lambda x: x - sched_date + pd.Timedelta(days=1))
    .apply(lambda x: x.total_seconds() / 60)
)

data.drop(
    columns=data.filter(regex="^(STT|NGÀY TỚI HẠN|NGÀY ĐIỀU ĐỘ|Unnamed.*)").columns,
    inplace=True,
)
data.rename(
    columns={
        "MÃ HÀNG": "id",
        "THÀNH PHẦN": "component",
        "SẢN LƯỢNG (M)": "amount",
        "QUY TRÌNH GIA CÔNG": "sequence",
    },
    inplace=True,
)
data


,id,component,amount,sequence,deadline
0,2264WP4021,W50P37F13,1703,"[4, 3, 6, 5, 9, 10]",57600.0
1,2233RU2124(2),P69R29PU2,1500,"[1, 2, 4, 5, 3, 6, 8]",57600.0
2,"2233PT2134(1),(2)",W50P50,5565,"[3, 5, 0, 4, 6, 9, 8, 7]",59040.0
3,2247PU3040(3),W30P70,2000,"[3, 5, 0, 4, 6, 8, 9]",60480.0
4,2247PU3031(5),W30P70,2500,"[3, 5, 0, 4, 6, 9]",61920.0
5,2239WP2140,W51P43PU6,5568,"[0, 2, 4, 9, 5, 3, 6, 7, 8]",63360.0
6,DUC93324S1W2#10B,W50P37F13,11000,"[3, 5, 0, 4, 6, 7, 8, 10]",63360.0
7,2233RU2101,P69R29PU2,500,"[4, 5, 0, 6, 3, 7, 8, 10]",63360.0
8,2160PU4006(4),W50P50,9370,"[3, 5, 0, 4, 6, 8, 7]",64800.0
9,2160PU4005(1)(2),W50P50,15330,"[3, 5, 0, 4, 6, 9, 8, 7]",64800.0


In [36]:
with open("machines.json", mode="+w") as file:
    file.write(machines.to_json(index=True))
with open("input.json", mode="+w") as file:
    file.write(data.to_json(index=True))